In [1]:
import pandas as pd
from tqdm import tqdm
import os

/Users/fernadavo/.virtualenvs/dev_new/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/Users/fernadavo/.virtualenvs/dev_new/lib/python3.7/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  if __name__ == '__main__':


### Get the data from the web

In [ ]:
#!apt-get install p7zip-full p7zip-rar
# !pip install matplotlib

In [8]:
for year in tqdm([2023,2022,2021,2020,2019]):
    if year == 2023:
        i2m = list(zip(range(1,5), ['Gener','Febrer','Marc','Abril','Maig']))
    else:
        i2m = list(zip(range(1,13), ['Gener','Febrer','Marc','Abril','Maig','Juny','Juliol','Agost','Setembre','Octubre','Novembre','Desembre']))
    for month, month_name in tqdm(i2m):
        os.system(f"wget 'https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
        os.system(f"7z x '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
        # os.system(f"rm '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")

In [ ]:
os.system("wget 'https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/2022_04_Abril_BicingNou_INFORMACIO.7z'")
os.system("7z x '2022_04_Abril_BicingNou_INFORMACIO.7z'")
os.system("rm '2022_04_Abril_BicingNou_INFORMACIO.7z'")

info = pd.read_csv('2022_04_Abril_BicingNou_INFORMACIO.csv')

### Aggregate Data at hour level

In [53]:
data = pd.DataFrame()
for year in tqdm([2019,2020, 2023,2022,2021]):
    if year == 2023:
        i2m_2 = list(zip(range(1,5), ['Gener','Febrer','Marc','Abril','Maig']))
    elif year == 2019:
        i2m_2 = list(zip(range(3,11), ['Març','Abril','Maig','Juny','Juliol','Agost','Setembre','Octubre','Novembre','Desembre']))
    elif year == 2020:
        i2m_2 = list(zip(range(1,13), ['Gener','Febrer','Marc','Abril','Maig','Juny','Juliol','Agost','Setembre','Octubre','Novembre','Desembre']))
    else:
        i2m_2 = list(zip(range(1,13), ['Gener','Febrer','Març','Abril','Maig','Juny','Juliol','Agost','Setembre','Octubre','Novembre','Desembre']))
    for month, month_name in tqdm(i2m_2):
        df = pd.read_csv(f'{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.csv')
        df['last_reported_time'] = pd.to_datetime(df['last_reported'], unit='s')
        df['last_reported_time_hour'] = df['last_reported_time'].apply(lambda x: str(x).split(":")[0])
        df = df.groupby(by=['station_id','last_reported_time_hour']).first().reset_index()
        data = pd.concat([data, df])

data.to_csv('global_data.csv',compression='gzip')

from google.colab import files
files.download('global_data.csv')

/Users/fernadavo/.virtualenvs/dev_new/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
